In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains.history_aware_retriever import create_history_aware_retriever
import pickle
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# from utils.prompts import  contextualize_q_prompt, qa_prompt
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [3]:
vectorstore = FAISS.load_local("embedded_knowledge_base",embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0.1)


In [4]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [6]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

question = "tell me about nikles technologies for finishes"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
# chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg_1['answer'])])



In [7]:
chat_history

[HumanMessage(content='tell me about nikles technologies for finishes'),
 AIMessage(content='Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.')]

In [8]:
second_question = "tell me about the second one"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

I don't have specific information on the second finish from Nikles Technologies. If you are interested or have any questions about their finishes, please contact them via email at info@nikles.com for more details.


In [9]:
from langchain_community.utilities import SQLDatabase
mysql_uri = 'mysql+mysqlconnector://root:@127.0.0.1:3306/products'
db = SQLDatabase.from_uri(mysql_uri)

In [10]:
from langchain_core.prompts import ChatPromptTemplate

query_maker_template = """Based on the table schema and some chat history below, write a SQL query that would answer the user's question:

{schema}
{chat_history}
Question: {input}
SQL Query:"""

query_maker_prompt = ChatPromptTemplate.from_template(query_maker_template)

In [11]:
def get_schema(_):
    schema = db.get_table_info()
    return schema

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.05)

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | query_maker_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)
query_based_NL_template = """Based on the table schema below, question, sql query, and sql response and chat history, write a natural language response. DO NOT ever 
mention the table name in the question or response. DO NOT mention the sql query. Act like the query is for a shop that produces some products and you are the owner.
Be consciese and accurate in your response.

{schema}

{chat_history}

Question: {input}
SQL Query: {query}
SQL Response: {response}"""
# prompt_response = ChatPromptTemplate.from_template(template)
# query_based_NL_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", query_based_NL_template),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )
query_based_NL_prompt = ChatPromptTemplate.from_template(query_based_NL_template)

In [12]:
def run_query(query):
    return db.run(query)

sql_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | query_based_NL_prompt
    | llm
)

# full_chain = (
#     RunnablePassthrough.assign(query=sql_chain).assign(
#         schema=get_schema,
#         response=lambda vars: run_query(vars["query"]),
#     )
#     | prompt_response
#     | llm
# )
# def sql_qery(input):
#     return full_chain.invoke({"question": input})


In [13]:
third_question = "provide names of relevant products"
ai_msg_3 = sql_chain.invoke({"input": third_question,"chat_history":chat_history})


In [29]:
ai_msg_3.content

'We have a variety of products related to Nikles technologies for finishes, such as shower heads, hand showers, kitchen sprays, shower kits, and more. These products offer different features and styles to cater to various preferences and needs.'

In [15]:
chat_history

[HumanMessage(content='tell me about nikles technologies for finishes'),
 AIMessage(content='Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.')]

In [16]:
router_prompt_template = """Given the user question and chat history below, classify it as either being about `database`,or `pdf`. do not respond with more than one word.
you are given the schema of the only table in the database as well. Decide if the user question can be answered by looking at the database, if not classify as `pdf`.

remember these rules

- DO NOT respond with more than one word.
- if the input is about warranties, then classify it as `pdf`
- if the input is about products, then classify it as `database`
- if the input is about details of product types, then classify it as `pdf`
- if the input is about details of products, then classify it as `database`

Here is the schema
{schema}

<question>
{input}
</question>
{chat_history}
Classification:"""

router_prompt = ChatPromptTemplate.from_template(router_prompt_template)

router = (
    RunnablePassthrough.assign(schema=get_schema)
    | router_prompt
    |llm
    |StrOutputParser()
)


In [17]:
third_question = "provide names of relevant products"
router.invoke({"input": third_question,"chat_history":chat_history})

'database'

In [18]:
chat_history

[HumanMessage(content='tell me about nikles technologies for finishes'),
 AIMessage(content='Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.')]

In [19]:
chat_history.extend([HumanMessage(content=third_question), ai_msg_3])

In [20]:
import json

file_path = "nikles_finishes.json"

# Save the list to a JSON file
with open(file_path, 'w') as file:
    json.dump(chat_history, file)

print(f"Data has been saved to {file_path}")

# Define the file path
file_path = "history.json"

# Load the list from the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

print("Data retrieved from the file:")
print(data)


TypeError: Object of type HumanMessage is not JSON serializable

In [21]:
# Function to dump chat history to a file
def dump_chat_history(chat_history, filename):
    with open(filename, 'wb') as f:
        pickle.dump(chat_history, f)

# Dump the chat history to a file
dump_chat_history(chat_history, 'chat_history.pkl')

In [22]:
# Function to load chat history from a file
def load_chat_history(filename):
    with open(filename, 'rb') as f:
        chat_history = pickle.load(f)
    return chat_history

# Load the chat history from the file
loaded_chat_history = load_chat_history('chat_history.pkl')

# Verify the loaded chat history
for message in loaded_chat_history:
    print(f"{type(message).__name__}: {message.content}")

    # Function to load chat history from a file
def load_chat_history(filename):
    with open(filename, 'rb') as f:
        chat_history = pickle.load(f)
    return chat_history

# Load the chat history from the file
loaded_chat_history = load_chat_history('chat_history.pkl')

# Verify the loaded chat history
for message in loaded_chat_history:
    print(f"{type(message).__name__}: {message.content}")



HumanMessage: tell me about nikles technologies for finishes
AIMessage: Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.
HumanMessage: provide names of relevant products
AIMessage: We have a variety of products related to Nikles technologies for finishes, such as shower heads, hand showers, kitchen sprays, shower kits, and more. These products offer different features and styles to cater to various preferences and needs.
HumanMessage: tell me about nikles technologies for finishes
AIMessage: Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes ar

In [24]:
loaded_chat_history

[HumanMessage(content='tell me about nikles technologies for finishes'),
 AIMessage(content='Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.'),
 HumanMessage(content='provide names of relevant products'),
 AIMessage(content='We have a variety of products related to Nikles technologies for finishes, such as shower heads, hand showers, kitchen sprays, shower kits, and more. These products offer different features and styles to cater to various preferences and needs.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 4280, 'total_tokens': 4327}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cceb3d09-3481-4261-baf3-948606ab1355-0')]

In [25]:
loaded_chat_history.extend([HumanMessage(content=third_question), ai_msg_3])
loaded_chat_history

[HumanMessage(content='tell me about nikles technologies for finishes'),
 AIMessage(content='Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathrooms with a range of stylish options. The finishes are designed to be durable and sustainable, catering to a global customer base.'),
 HumanMessage(content='provide names of relevant products'),
 AIMessage(content='We have a variety of products related to Nikles technologies for finishes, such as shower heads, hand showers, kitchen sprays, shower kits, and more. These products offer different features and styles to cater to various preferences and needs.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 4280, 'total_tokens': 4327}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cceb3d09-3481-4261-baf3-948606ab1355-0'),
 Hu

In [28]:
# dump_chat_history(loaded_chat_history, "chat_history")
reloaded_ch=load_chat_history("chat_history1.pkl")
reloaded_ch

FileNotFoundError: [Errno 2] No such file or directory: 'chat_history1.pkl'

In [ ]:
from langchain_core.load import *
x = dumps(chat_history,'chat_history.json')

TypeError: dumps() takes 1 positional argument but 2 were given

In [ ]:
y= loads(chat_history)

c:\Users\BS-Test\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


TypeError: the JSON object must be str, bytes or bytearray, not list

In [ ]:
x

'[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "tell me about nikles technologies for finishes", "type": "human"}}, "Nikles offers Luxury Finishes in Brushed Gold, Polished Gold, Rose Gold, Brushed Nickel, Gun Metal, and Matte Black. These finishes allow customers to add a modern touch to their bathroom with a variety of color options that are durable and sustainable. The finishes are designed to provide a noble shine and a modern look to the bathroom.", {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "provide names of relevant products", "type": "human"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "AIMessage"], "kwargs": {"content": "The relevant products related to Nikles technologies for finishes are \\"SHOWER HEAD TRONICO 250\\", \\"LIPS CASCADE SHOWER HEAD - MATTE BLACK\\", and \\"LIPS CASCADE SHOWER HEAD - RED\\"."